In [ ]:
import os
import numpy as np
import pandas as pd
from google.cloud import bigquery

In [ ]:
PROJECT_ID = ""

if not os.getenv("IS_TESTING"):
    # Get your Google Cloud project ID from gcloud
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

In [ ]:
collection_ID = "lctsc"
bq_client = bigquery.Client(PROJECT_ID)

selection_query = f"\
  WITH studies AS (SELECT DISTINCT StudyInstanceUID FROM `bigquery-public-data.idc_current.dicom_all` \
                  WHERE collection_id =  \"{collection_ID}\"  \
                    LIMIT 10) \
  SELECT  \
    StudyInstanceUID, \
    SeriesInstanceUID, \
    SOPInstanceUID, \
    gcs_url \
  FROM \
    `bigquery-public-data.idc_current.dicom_all` INNER JOIN studies USING(StudyInstanceUID) \
    "

selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

In [ ]:
pd.set_option('display.max_colwidth', None)

selection_df

In [ ]:
# save the list of files
# save the list of GCS URLs into a file
import os
idc_download_folder = "./IDC_downloads"
if not os.path.exists(idc_download_folder):
  os.mkdir(idc_download_folder)

selection_manifest = os.path.join(idc_download_folder, "lctsc_manifest.txt")
selection_df["gcs_url"].to_csv(selection_manifest, header=False, index=False)

In [ ]:
# create a staging bucket
bucket_name = "gs://"+PROJECT_ID+"_lab1"
print("Bucket Name: ",bucket_name)
stage_dir = bucket_name+"/LCTSC"
print("Staging Directory: ",stage_dir)

In [ ]:
#create the bucket
!gsutil mb $bucket_name

In [ ]:
# copy files to staging area
# download is this simple!
!cat ./IDC_downloads/lctsc_manifest.txt | gsutil -m cp -I $stage_dir 2> copy.log

In [ ]:
!gsutil ls $stage_dir | wc -l